In [240]:
import fusion_gym_environment
import fusion_gym.fusion_env_wrapper 

import importlib
import os 
import sys
import uuid

from tf_agents.environments import utils

import numpy as np

importlib.reload(fusion_gym_environment)
importlib.reload(fusion_gym.fusion_env_wrapper)

##############################
##### Fusion gym imports #####
##############################

# Add the client folder to sys.path
CLIENT_DIR = os.path.join("..", "tools", "fusion360gym", "client")
if CLIENT_DIR not in sys.path:
    sys.path.append(CLIENT_DIR)

from gym_env import GymEnv

In [241]:
action = fusion_gym_environment.ExtrudeAction(
    uuid.UUID(face1),
    uuid.UUID(face2),
    fusion_gym_environment.Operation.CutFeatureOperation
)

encoded = action.encode()
decoded = fusion_gym_environment.ExtrudeAction.decode(encoded)
assert decoded == action

In [242]:
# Setup Gym client
gym_env = FusionEnvWrapper(host="127.0.0.1", port=8080, launch_gym=False)

In [243]:
environment = fusion_gym_environment.FusionGymEnvironment(gym_env, 5)
utils.validate_py_environment(environment, episodes=5)

step: State(step=1, finished=False)
step: State(step=2, finished=False)
step: State(step=3, finished=False)
step: State(step=4, finished=False)
step: State(step=5, finished=False)
step: State(step=6, finished=True)
step: State(step=1, finished=False)
step: State(step=2, finished=False)
step: State(step=3, finished=False)
step: State(step=4, finished=False)
step: State(step=5, finished=False)
step: State(step=6, finished=True)
step: State(step=1, finished=False)
step: State(step=2, finished=False)
step: State(step=3, finished=False)
step: State(step=4, finished=False)
step: State(step=5, finished=False)
step: State(step=6, finished=True)
step: State(step=1, finished=False)
step: State(step=2, finished=False)
step: State(step=3, finished=False)
step: State(step=4, finished=False)
step: State(step=5, finished=False)
step: State(step=6, finished=True)
step: State(step=1, finished=False)
step: State(step=2, finished=False)
step: State(step=3, finished=False)
step: State(step=4, finished=Fal

In [101]:
import json
import os
from pathlib import Path
from typing import List

def get_files(input_dir: str, split_file: str = None) -> List[str]:
    """ Get list of .smt files from the target directory 
    input_dir -- the directory input files
    split -- the path of the test file
    """
    input_dir = Path(os.path.expanduser(input_dir))
    
    if not input_dir.exists():
        print("Input file/folder does not exist")
        return []
        
    test_files = None
    if split_file is not None:
        split_file = Path(os.path.expanduser(split_file))
        if not split_file.exists():
            print("Split file does not exists")
        else:
            with open(split_file, encoding="utf8") as f:
                json_data = json.load(f)
                if "test" not in json_data:
                    print("Split file does not have a test set")
                else:
                    test_files = set()
                    for test_file in json_data["test"]:
                        test_files.add(f"{test_file}.smt")

    files = []
    if input_dir.is_dir():
        smt_files = [f for f in input_dir.glob("**/*.smt")]
        if len(smt_files) == 0:
            print("No .smt files found")
            exit()
        else:
            # We store an index with the file for debug output
            for smt_file in smt_files:
                if test_files is None:
                    # If we don't have a split
                    # use all files
                    files.append(smt_file)
                else:
                    # If we have a split
                    # use only the test files
                    if smt_file.name in test_files:
                        files.append(smt_file)
    else:
        files.append(input)
    return files

In [95]:
INPUT_DIR = "~/Downloads/r1.0.0/reconstruction/"
input_files = get_files(INPUT_DIR)

In [141]:
first_input_file

PosixPath('/Users/erikbeerepoot/Downloads/r1.0.0/reconstruction/68475_17ef1725_0000_0035.smt')

In [ ]:
first_input_file = input_files[0]
gym_env.set_target(first_input_file)

In [142]:
face1 = "850e641c-e309-11ea-ad98-54bf646e7e1f"
face2 = "850eb246-e309-11ea-a0ba-54bf646e7e1f"

In [244]:
action = fusion_gym_environment.ExtrudeAction(
    uuid.UUID(face1),
    uuid.UUID(face2),
    fusion_gym_environment.Operation.CutFeatureOperation
)

In [245]:
environment.step(action.encode())

step: State(step=1, finished=False)
action: ExtrudeAction(start_face=UUID('850e641c-e309-11ea-ad98-54bf646e7e1f'), end_face=UUID('850eb246-e309-11ea-a0ba-54bf646e7e1f'), operation=<Operation.CutFeatureOperation: 2>)
Invalid start_face value
None


TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([1], dtype=int32))